In [5]:
import redis, json, time, operator

## 建立連線與向資料庫倒入資料

In [3]:
'''
建立連線，輸入相關參數即可
'''
def connect4Input(host, port, db):
    connection = redis.Redis(host = host, port = port, db = db)
    print 'Connection: %s' % True
    return connection

'''
呼叫pipeline實體化，使用transation，
故使用mulit這個方法，如果執行上沒有錯誤，
接著就透過execute這個方法送出資料
如果發生錯誤，繼續使用continue使資料倒進資料庫不會中斷
'''
def ready2Input(connection, result):
    for tourAgency in result.keys():
        with connection.pipeline(transaction = True) as pipe:
            tourAgencyCount = 0
            try:
                pipe.watch(tourAgency)
                tourAgencyCount += 1
                pipe.multi()
                connection.hmset(tourAgency, result[tourAgency])
                pipe.execute()
            except redis.WatchError:
                continue

## 執行

In [6]:
path = r'D:\BigData\Python\Workspace\TextMining\Project\txtFiles\totalAtriclesResult\test.json'
with open(path) as dataRead:
    start = time.time()
    result = json.load(dataRead)
    connection = connect4Input(host = '36.236.58.48', port = 6379, db = 0)
    ready2Input(connection, result)
    end = time.time()
    print 'Data is finished to input in Redis!!'
    print 'Time Cost: %f sec.' % (end - start)

Connection: True
Data is finished to input in Redis!!
Time Cost: 3.819000 sec.


## Query

In [8]:
'''
取得連線，並回傳連線物件
'''
def connect4Query(host, port, db):
    myServer = redis.Redis(host = host, port = port, db = db)
    print 'Connection: %s' % True
    return myServer

'''
外部來的參數有3個，第1個是取得連線物件，
第2個是要查詢的旅行社名稱，
第3個則是要取前N個的數量。
接著將取出的word_count做資料型態轉換，
Redis中，所有key,Value都會變成Str，
所以，取出時做排序前，必須先轉換成Int，
最後回傳前N個值(包在List中)
'''
def changeType2SortData(myServer, tourAgency, topn):
    new_result = list()
    result = myServer.hscan_iter(tourAgency)
    for count_word, word_count in result: #eval(word_count) == int(word_count)
        new_result.append((count_word.decode('utf-8'), eval(word_count)))
    new_result = sorted(new_result, key = operator.itemgetter(1), reverse = True)
    return new_result[0:topn]

'''
實際執行的方法，
取得連線後，再輸入要查詢的名稱，
為了方便前端人員的作業，
所以再將原本tuple的型態，
一一取出，轉換成k,v的樣子。
並且依序放進list中。
'''
def start():
    textRank = list()
    tourAgency = raw_input(u'請輸入你要查詢的旅行社: ')
    myServer = connect4Query(host = '36.236.58.48', port = 6379, db = 0)
    new_result = changeType2SortData(myServer, tourAgency, 15)
    for count_word, word_count in new_result:
        textRank.append({'word':count_word, 'wordcount':word_count})
    return textRank

In [11]:
start()

請輸入你要查詢的旅行社: phoenix
Connection: True


[{'word': u'\u512a\u60e0', 'wordcount': 226},
 {'word': u'\u597d', 'wordcount': 127},
 {'word': u'\u63a8\u85a6', 'wordcount': 115},
 {'word': u'\u4e0d\u932f', 'wordcount': 62},
 {'word': u'\u4fbf\u5b9c', 'wordcount': 55},
 {'word': u'\u4e0d\u597d', 'wordcount': 28},
 {'word': u'\u5c08\u696d', 'wordcount': 28},
 {'word': u'\u8cb4', 'wordcount': 27},
 {'word': u'\u5f88\u597d', 'wordcount': 22},
 {'word': u'\u5f88\u4e0d\u932f', 'wordcount': 21},
 {'word': u'\u6bd4\u8f03\u597d', 'wordcount': 19},
 {'word': u'\u5dee', 'wordcount': 18},
 {'word': u'\u9ebb\u7169', 'wordcount': 18},
 {'word': u'\u5f88\u5dee', 'wordcount': 16},
 {'word': u'\u6700\u597d', 'wordcount': 16}]